# Tutorial for the Richardson-Gaudin model

In [1]:
import numpy as np
from moha.hamiltonians import HamRG, HamHeisenberg
from pyscf import fci

# Define the connectivity matrix
connectivity = np.array([[1, 1, 1, 1],
                         [1, 1, 1, 1],
                         [1, 1, 1, 1],
                         [1, 1, 1, 1]])

# connectivity = np.array([[1, 1],
#                          [1, 1]])


# Define the Hamiltonian parameters
J = -3
J_eq = J * connectivity
mu = J * np.ones(4)

# Define the Hamiltonian
ham = HamRG(J_eq=J_eq, mu=mu)
e0 = ham.generate_zero_body_integral()

h1 = ham.generate_one_body_integral(dense=True, basis='spatial basis')
h2 = ham.generate_two_body_integral(dense=True, basis='spatial basis', sym=4)

h2_ch = np.einsum('ijkl->ikjl', h2)


n_sites = 4
occ = (n_sites//2, n_sites//2)
# e0 += -3 * 2/2

e, ci = fci.direct_spin0.kernel(h1, 2*h2_ch, nelec=occ, norb=n_sites, nroots=6, ecore=e0)


print(e, e0)

# compute rdms
rdm1, rdm2 = fci.direct_spin0.make_rdm12(ci[0], norb=n_sites, nelec=occ)

# compute the repulsion energy
E_rep = e[0] - np.sum(h1*rdm1)
print("total energy: ", e)
print("repulsion energy: ", E_rep)

KeyboardInterrupt: 

In [ ]:
h1

matrix([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [ ]:
for i, j, k, l in zip(*np.where(h2 != 0)):
    elem = h2[i, j, k, l]
    print(i, j, k, l,": ", elem)

print("one-body integral: ", h1)


0 0 0 0 :  -1.5
0 0 1 1 :  -1.5
0 0 2 2 :  -1.5
0 0 3 3 :  -1.5
1 1 0 0 :  -1.5
1 1 1 1 :  -1.5
1 1 2 2 :  -1.5
1 1 3 3 :  -1.5
2 2 0 0 :  -1.5
2 2 1 1 :  -1.5
2 2 2 2 :  -1.5
2 2 3 3 :  -1.5
3 3 0 0 :  -1.5
3 3 1 1 :  -1.5
3 3 2 2 :  -1.5
3 3 3 3 :  -1.5
one-body integral:  [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [4]:
import ast
with open('../rg_1dm/pf_63.dat') as f:
    data = f.readlines()

eps = ', '.join(data[0].split())
eps = np.array(ast.literal_eval(eps))


In [12]:
import re
with open('../rg_1dm/pf_63.dat') as f:
    data = f.read()

parsed = []
pattern = r"BCS Energy:\s*\n(-?\d+\.\d+)\s+\[([01](?:,\s*[01])*)\]\s+(-?\d+\.\d+)"
matches = re.findall(pattern, data)
for g_str, occ_str, energy_str in matches:
    g = float(g_str)
    occupation = [int(x.strip()) for x in occ_str.split(',')]
    energy = float(energy_str)
    parsed.append((g, occupation, energy))

with open('../rg_1dm/pf_63.dat') as f:
    data = f.readlines()
eps = ', '.join(data[0].split())
eps = np.array(ast.literal_eval(eps))

In [ ]:
print("eps:", eps)
print("parsed data:", parsed[:10])
parsed = np.array(parsed)
mask_1 = [x[1] == [1, 1, 1, 0, 0, 0] for x in parsed]
mask_2 = [x[0] > 0 for x in parsed]
mask = np.logical_and(mask_1, mask_2)

parsed_filtered = parsed[mask]


eps: [0. 1. 2. 3. 4. 5.]
parsed data: [(-1000.0, [1, 1, 1, 0, 0, 0], 7.48795238), (-990.010101010101, [1, 1, 1, 0, 0, 0], 7.48783081), (-980.0202020202021, [1, 1, 1, 0, 0, 0], 7.48770677), (-970.030303030303, [1, 1, 1, 0, 0, 0], 7.48758017), (-960.040404040404, [1, 1, 1, 0, 0, 0], 7.48745093), (-950.0505050505051, [1, 1, 1, 0, 0, 0], 7.48731898), (-940.060606060606, [1, 1, 1, 0, 0, 0], 7.48718423), (-930.070707070707, [1, 1, 1, 0, 0, 0], 7.48704658), (-920.0808080808081, [1, 1, 1, 0, 0, 0], 7.48690594), (-910.0909090909091, [1, 1, 1, 0, 0, 0], 7.48676221)]


/var/folders/gf/7vfqp7nn2972hrcz727tkjkh0000gn/T/ipykernel_29648/405346563.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  parsed = np.array(parsed)


In [30]:
g_lst = parsed_filtered[:, 0]
energy_lst = parsed_filtered[:, 2]

for g, energy in zip(g_lst, energy_lst):
    print(f"g: {g}, energy: {energy}")

g: 0.001, energy: 2.99849907
g: 0.10200000000000001, energy: 2.83648338
g: 0.203, energy: 2.64993049
g: 0.30400000000000005, energy: 2.43241813
g: 0.405, energy: 2.17779335
g: 0.506, energy: 1.88201527
g: 0.6070000000000001, energy: 1.54451998
g: 0.7080000000000001, energy: 1.16805883
g: 0.809, energy: 0.75734879
g: 0.91, energy: 0.31764826
g: 1.011, energy: -0.14612126
g: 1.1119999999999999, energy: -0.62972814
g: 1.213, energy: -1.12968138
g: 1.314, energy: -1.64315444
g: 1.415, energy: -2.16787376
g: 1.516, energy: -2.70201062
g: 1.617, energy: -3.24408888
g: 1.718, energy: -3.79291051
g: 1.819, energy: -4.34749702
g: 1.92, energy: -4.90704382
g: 2.021, energy: -5.47088478
g: 2.122, energy: -6.03846472
g: 2.223, energy: -6.60931789
g: 2.324, energy: -7.18305117
g: 2.4250000000000003, energy: -7.75933071
g: 2.5260000000000002, energy: -8.33787143
g: 2.6270000000000002, energy: -8.9184286
g: 2.728, energy: -9.50079104
g: 2.829, energy: -10.08477566
g: 2.93, energy: -10.670223
g: 3.031

In [1]:
import numpy as np
from moha.hamiltonians import HamRG, HamHeisenberg
from pyscf import fci

# Define the connectivity matrix
connectivity = np.array([[1, 1, 1, 1, 1, 1],
                         [1, 1, 1, 1, 1, 1],
                         [1, 1, 1, 1, 1, 1],
                         [1, 1, 1, 1, 1, 1],
                         [1, 1, 1, 1, 1, 1],
                         [1, 1, 1, 1, 1, 1]])
exact = -3.24408888
g_pj = 1.617
# Define the Hamiltonian parameters
J = -g_pj / 2
J_eq = J * connectivity

eps = np.array([0., 1., 2., 3., 4., 5.])
mu = eps + J
# Define the Hamiltonian
ham = HamRG(J_eq=J_eq, mu=mu)
e0 = ham.generate_zero_body_integral()

h1 = ham.generate_one_body_integral(dense=True, basis='spatial basis')
h2 = ham.generate_two_body_integral(dense=True, basis='spatial basis', sym=4)

h2_ch = np.einsum('ijkl->ikjl', h2)


n_sites = 6
occ = (n_sites//2, n_sites//2)
# e0 += -3 * 2/2

e, ci = fci.direct_spin0.kernel(1*h1, 2*h2_ch, nelec=occ, norb=n_sites, nroots=6, ecore=0)


print(e, e0)

# compute rdms
rdm1, rdm2 = fci.direct_spin0.make_rdm12(ci[0], norb=n_sites, nelec=occ)

# compute the repulsion energy
E_rep = e[0] - np.sum(h1*rdm1)
print("total energy: ", e)
print("exact energy: ", exact)
print("repulsion energy: ", E_rep)

[-3.24408888  0.93791546  1.52629165  1.52629165  2.30351616  2.30351616] -7.5
total energy:  [-3.24408888  0.93791546  1.52629165  1.52629165  2.30351616  2.30351616]
exact energy:  -3.24408888
repulsion energy:  -8.735475966052606


In [81]:
e0

-7.5

In [80]:
mu

array([-5.0000e-04,  9.9950e-01,  1.9995e+00,  2.9995e+00,  3.9995e+00,
        4.9995e+00])

In [78]:
h1

matrix([[0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0.5, 0. , 0. , 0. , 0. ],
        [0. , 0. , 1. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 1.5, 0. , 0. ],
        [0. , 0. , 0. , 0. , 2. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 2.5]])

In [79]:
non_zero_indices = np.nonzero(h2)
for i, j, k, l in zip(*non_zero_indices):
    elem = h2[i, j, k, l]
    print(i, j, k, l,": ", elem)

0 0 0 0 :  -0.00025
0 0 1 1 :  -0.00025
0 0 2 2 :  -0.00025
0 0 3 3 :  -0.00025
0 0 4 4 :  -0.00025
0 0 5 5 :  -0.00025
1 1 0 0 :  -0.00025
1 1 1 1 :  -0.00025
1 1 2 2 :  -0.00025
1 1 3 3 :  -0.00025
1 1 4 4 :  -0.00025
1 1 5 5 :  -0.00025
2 2 0 0 :  -0.00025
2 2 1 1 :  -0.00025
2 2 2 2 :  -0.00025
2 2 3 3 :  -0.00025
2 2 4 4 :  -0.00025
2 2 5 5 :  -0.00025
3 3 0 0 :  -0.00025
3 3 1 1 :  -0.00025
3 3 2 2 :  -0.00025
3 3 3 3 :  -0.00025
3 3 4 4 :  -0.00025
3 3 5 5 :  -0.00025
4 4 0 0 :  -0.00025
4 4 1 1 :  -0.00025
4 4 2 2 :  -0.00025
4 4 3 3 :  -0.00025
4 4 4 4 :  -0.00025
4 4 5 5 :  -0.00025
5 5 0 0 :  -0.00025
5 5 1 1 :  -0.00025
5 5 2 2 :  -0.00025
5 5 3 3 :  -0.00025
5 5 4 4 :  -0.00025
5 5 5 5 :  -0.00025
